# **Metriche**
## *Metriche di clustering*
Definizione della metriche per il clustering: *adjusted rand index* ed il *silhouette score*.

## **Adjusted Rand Index**

Sia $C$ l'insieme dei cluster "veri" assegnati ad un certo dataset, e $K$ l'insieme dei cluster assegnati a valle dell'applicazione di un algoritmo di clustering. Allora definiamo l'**Indice di Rand** come:

$$RI=\frac{a+b}{C_2^n}$$

dove:

- $a$ è il numero di coppie di campioni che appartengono allo stesso cluster sia in $C$ sia a valle dell'assegnazione $K$;

- $b$ è il numero di coppie di campioni che non appartengono allo stesso cluster sia in $C$ sia a valle dell'assegnazione $K$;

- $C_2^n$ è il numero totale di coppie di campioni presenti nel dataset.

Si può dimostrare non è garantito che l'indice di Rand assuma valore vicino allo zero a seguito di un'assegnazione completamente casuale dei cluster da parte dell'algoritmo.

Possiamo quindi tenere conto dell'aspettazione $E[RI]$ di ottenere un'assegnazione casuale mediante l'*indice di Rand modificato*:

$$ARI=\frac{RI-E[RI]}{max(RI)-E[RI]}$$

In **Scikit Learn**, l'indice di Rand modificato è ottenuto usando la funzione `adjusted_rand_score() ` del package metrics.

Il valore ottimale dell'ARI è pari proprio ad 1, caso in cui il clustering è riuscito a predire correttamente tutte le classi dei singoli campioni. Valori prossimi allo zero o negativi (fino a -1) contraddistinguono invece labeling non corretti.

Una metrica di questo tipo ha l'ovvio vantaggio di essere facilmente interpretabile, oltre che di non essere collegata ad uno specifico algoritmo di clustering.

Tuttavia, vi è una criticità indotta dalla necessità di **conoscere a priori il labeling** esatto dei campioni (il che, quindi, potrebbe farci propendere per l'uso di un algoritmo di classificazione).

## **Silhoutte Score**

A differenza dell'**ARI**, il **silhouette score** non richiede la *conoscenza aprioristica* delle label vere; per valutare la qualità del clustering, invece, questa metrica si affida a valutazioni sulla separazione dei cluster, ottenendo un valore tanto più alto quanto questi sono tra di loro ben separati e definiti.

In particolare, il silhouete score per un singolo campione è definito come:

$$s=\frac{b-a}{max(a,b)}$$

dove:

- $a$ è la distanza media tra un campione e tutti gli altri campioni appartenenti allo stesso cluster;
- $b$ è la distanza media tra un campione e tutti gli altri campioni appartenenti al cluster più vicino.

Questa metrica, implementata grazie alla funzione `silhouette_score()` del package `metrics`, è anch'essa di facile interpretazione, in quanto può assumere valori compresi nell'intervallo $[-1,1]$, con:

- valori prossimi a $-1$ che indicano un clustering non corretto;
- valori prossimi allo $0$ che indicano cluster sovrapposti;
- valori prossimi a $+1$ che indicano cluster densi e ben suddivisi.

Uno svantaggio del silhouette score è che, in generale, può variare in base all'algoritmo utilizzato.

## **Esercizi**

In [7]:
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from sklearn.datasets import make_blobs
from sklearn.metrics import adjusted_rand_score, silhouette_score

### **Es 7.0**

Ricreiamo le condizioni sperimentali degli esercizi *Es 6.3* ed *Es 6.2*.

Stavolta, però, valutiamo le performance di ogni algoritmo utilizzando l'ARI ed il silhouette score.

Inoltre, proviamo a vedere cosa accade per i seguenti parametri:

- per il K-Means, facciamo variare il numero di cluster tra `2` e `5`;
- per il DBSCAN, assegnamo ad $\epsilon$ i valori `0.5` o `1.0`, ed a `min_samples` i valori `5` e `10`.

Per ognuno dei due algoritmi, infine, riportiamo a schermo solo i valori dei parametri per i quali le metriche assumono valore massimo.

In [8]:
def print_kmeans_results(X, y, use_case, verbose=False):
    print(f'{use_case} - KMeans')
    n_clusters_ari_best = 2
    n_clusters_silhouette_best = 2
    ari_best = 0
    silhouette_best = 0
    for n_clusters in range(2, 6):
        kmeans = KMeans(n_clusters=n_clusters)
        preds = kmeans.fit_predict(X)
        ari = round(adjusted_rand_score(y, preds), 2)
        silhouette = round(silhouette_score(X, preds), 2)
        if verbose:
            print(f'Numero di cluster: {n_clusters}')
            print(f'ARI KMeans: {ari}')
            print(f'Silhouette score KMeans: {silhouette}')
        if ari > ari_best:
            ari_best = ari
            n_clusters_ari_best = n_clusters
        if silhouette > silhouette_best:
            silhouette_best = silhouette
            n_clusters_silhouette_best = n_clusters
    print(
        f'''Parametri con il valore massimo di ARI:
        num clusters: {n_clusters_ari_best}
        Valore massimo di ARI: {ari_best}''')
    print(
        f'''Parametri con il valore massimo di silhouette:
        num clusters: {n_clusters_silhouette_best}
        Valore massimo di silhouette: {silhouette_best}''')


def print_dbscan_results(X, y, use_case, verbose=False):
    print(f'{use_case} - DBSCAN')
    eps_mins_ari_best = [0, 0]
    eps_mins_silhouette_best = [0, 0]
    ari_best = 0
    silhouette_best = 0
    for epsilon in [0.5, 1.0]:
        for min_samples in [5, 10]:
            dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
            preds = dbscan.fit_predict(X)
            ari = round(adjusted_rand_score(y, preds), 2)
            silhouette = round(silhouette_score(X, preds), 2)
            if verbose:
                print(f'Epsilon: {epsilon} - Min samples: {min_samples}')
                print(f'ARI KMeans: {ari}')
                print(f'Silhouette score KMeans: {silhouette}')
            if ari > ari_best:
                ari_best = ari
                eps_mins_ari_best = [epsilon, min_samples]
            if silhouette > silhouette_best:
                silhouette_best = silhouette
                eps_mins_silhouette_best = [epsilon, min_samples]
    print(
        f'''Parametri con il valore massimo di ARI:
        eps: {eps_mins_ari_best[0]} - min samples: {eps_mins_ari_best[1]}
        Valore massimo di ARI: {ari_best}''')
    print(
        f'''Parametri con il valore massimo di silhouette score:
        eps: {eps_mins_silhouette_best[0]} - min samples: {eps_mins_silhouette_best[1]}
        Valore massimo di silhouette: {silhouette_best}''')

In [9]:
X, y = make_blobs(n_samples=1000, random_state=42)

print_kmeans_results(X, y, 'Cluster corretti')
print_dbscan_results(X, y, 'Cluster corretti')

Cluster corretti - KMeans
Parametri con il valore massimo di ARI:
        num clusters: 3
        Valore massimo di ARI: 1.0
Parametri con il valore massimo di silhouette:
        num clusters: 3
        Valore massimo di silhouette: 0.84
Cluster corretti - DBSCAN
Parametri con il valore massimo di ARI:
        eps: 1.0 - min samples: 5
        Valore massimo di ARI: 0.99
Parametri con il valore massimo di silhouette score:
        eps: 1.0 - min samples: 5
        Valore massimo di silhouette: 0.82


In [10]:
# Ipotesi 1: anisotropia
t = np.tan(np.radians(60))
rot = np.array([[1, t], [0, 1]])
X_an = X.dot(rot)

print_kmeans_results(X_an, y, 'Cluster anisotropi')
print_dbscan_results(X_an, y, 'Cluster anisotropi')

Cluster anisotropi - KMeans
Parametri con il valore massimo di ARI:
        num clusters: 3
        Valore massimo di ARI: 0.99
Parametri con il valore massimo di silhouette:
        num clusters: 2
        Valore massimo di silhouette: 0.82
Cluster anisotropi - DBSCAN
Parametri con il valore massimo di ARI:
        eps: 1.0 - min samples: 5
        Valore massimo di ARI: 1.0
Parametri con il valore massimo di silhouette score:
        eps: 1.0 - min samples: 5
        Valore massimo di silhouette: 0.74


In [11]:
# Ipotesi 2: diversa varianza
X_var, y_var = make_blobs(
    n_samples=1000,
    random_state=200,
    cluster_std=[1.8, 2.5, 2.4])

print_kmeans_results(X_var, y_var, 'Cluster a diversa varianza')
print_dbscan_results(X_var, y_var, 'Cluster a diversa varianza')

Cluster a diversa varianza - KMeans
Parametri con il valore massimo di ARI:
        num clusters: 3
        Valore massimo di ARI: 0.75
Parametri con il valore massimo di silhouette:
        num clusters: 3
        Valore massimo di silhouette: 0.49
Cluster a diversa varianza - DBSCAN
Parametri con il valore massimo di ARI:
        eps: 0.5 - min samples: 5
        Valore massimo di ARI: 0.32
Parametri con il valore massimo di silhouette score:
        eps: 1.0 - min samples: 5
        Valore massimo di silhouette: 0.31


In [12]:
# Ipotesi 3: diversa cardinalità
X, y = make_blobs(n_samples=1000, random_state=12)
X_uneven = np.concatenate(
    (X[y == 0][:200], X[y == 1][:50], X[y == 2][:10]),
    axis=0)
y_uneven = np.concatenate(
    (y[y == 0][:200], y[y == 1][:50], y[y == 2][:10]),
    axis=0)

print_kmeans_results(X_var, y_var, 'Cluster a diversa cardinalità')
print_dbscan_results(X_var, y_var, 'Cluster a diversa cardinalità')

Cluster a diversa cardinalità - KMeans
Parametri con il valore massimo di ARI:
        num clusters: 3
        Valore massimo di ARI: 0.75
Parametri con il valore massimo di silhouette:
        num clusters: 3
        Valore massimo di silhouette: 0.49
Cluster a diversa cardinalità - DBSCAN
Parametri con il valore massimo di ARI:
        eps: 0.5 - min samples: 5
        Valore massimo di ARI: 0.32
Parametri con il valore massimo di silhouette score:
        eps: 1.0 - min samples: 5
        Valore massimo di silhouette: 0.31
